<h1>Notes From PostgreSQL Documentation</h1>

<h3>Installing Dependencies</h3>

<b>Install dependencies and after installment finished restart the notebook.</b>

In [ ]:
#!pip install jupyter ipython-sql jupyterlab_sql_editor[all]

<h3>Initializing Notebook and Connections</h3>

<b>This steps will be executed every start/restart of kernel or notebook</b>

In [ ]:
%load_ext sql

In [ ]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

Connecting to DB
Usage: &lt;db_vendor&gt;://[username:password]@&lt;serverip&gt;/&lt;dbname&gt;

In [ ]:
%sql postgresql://postgres@localhost/mydb

Testing connection

In [ ]:
%sql select 'CONNECTION SUCCESSFULLY INITIALIZED' as result

<h2>Chapter 5. Data Definition</h2>

### 5.1. Table Basics

##### a. Creating Table

For details check reference <a href="https://www.postgresql.org/docs/current/sql-createtable.html">https://www.postgresql.org/docs/current/sql-createtable.html</a>

In [ ]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric
);

##### b. Deleting Table

For details check reference <a href="https://www.postgresql.org/docs/current/sql-droptable.html">https://www.postgresql.org/docs/current/sql-droptable.html</a>

- IF EXISTS is optional. This statement prevents throwing error when table does not exist in DB.

In [ ]:
%%sql
DROP TABLE IF EXISTS products;

##### c. Creating Table with DEFAULT value

In [ ]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric DEFAULT 9.99,
	date timestamp DEFAULT CURRENT_TIMESTAMP
);

### 5.3. Identity Columns

##### a. Creating table with IDENTITY column

- When using <i>IDENTITY</i> column with <i>GENERATED</i> it means value of column will be created automatically
- <i>GENERATED ALWAYS</i> means you can not insert manual values when adding/updating for this field.
- <i>GENERATED BY DEFAULT</i> means you can insert manual values when adding/updating for this field
- When using <i>GENERATED BY DEFAULT</i> you must be careful:
    - Generating values will be using a sequential. That means it starts from 1 and goes on.
    - If identity column is described as UNIQUE:
        - You insert a record without inserting a manual value for identity column.
        - After that you give identity column value by hand and that value is 2.
            - It will be successfully inserted.
            - But sequence value is still remains as 2.
        - Then you inserted a record without inserting a manual value for identity column again.
            - Since sequence value is 2 and there is a record also has valued 2, it will throw duplicated value error.
    - If identity column is not described as UNIQUE:
        - You insert a record without inserting a manual value for identity column.
        - After that you give identity column value by hand and that value is 2.
            - It will be successfully inserted.
            - But sequence value is still remains as 2.
        - Then you inserted a record without inserting a manual value for identity column again.
            - Since sequence value is 2 it will insert record with identity column has value as 2.
            - But there are two record which their identity columns has value as 2.
- Even when using <i>GENERATED ALWAYS</i> there are duplicated value or error risks:
    - By using OVERRIDING SYSTEM VALUE, you can enter identity column values manually.
    - Same cases above (GENERATED BY DEFAULT) are also effective for this. 

##### b. Example usage of <i>GENERATED ALWAYS</i>

- Creating table with GENERATED ALWAYS AS IDENTITY column

In [ ]:
%%sql
DROP TABLE IF EXISTS people;
CREATE TABLE people (
	id bigint GENERATED ALWAYS AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

- Record will be inserted successfully and id will be automatically assigned as 1

In [ ]:
%%sql 
INSERT INTO people (name, address) VALUES ('A', 'foo');
SELECT * FROM people;

- Throws error because of id column is generated always and given an id value

In [ ]:
%sql INSERT INTO people (id, name, address) VALUES (2, 'B', 'bar');

- Will not throw error and insert record successfully

In [ ]:
%%sql 
INSERT INTO people (id, name, address) OVERRIDING SYSTEM VALUE VALUES (2, 'B', 'bar');

- Insert will be successful but since sequence for id value is still 2 and there is a record which id is 2, there is two record which has id 2.

In [ ]:
%%sql 
INSERT INTO people (name, address) VALUES ('C', 'baz');
SELECT * FROM people;

##### c. Example usage of <i>GENERATED BY DEFAULT with UNIQUE descriptor</i>

- Creating table with GENERATED BY DEFAULT AS IDENTITY column

In [ ]:
%%sql
DROP TABLE IF EXISTS people;
CREATE TABLE people (
	id bigint UNIQUE GENERATED BY DEFAULT AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

- First record will be inserted successfully and id will be automatically assigned as 1.
- Second record will be inserted successfully.

In [ ]:
%%sql 
INSERT INTO people (name, address) VALUES ('A', 'foo');
INSERT INTO people (id, name, address) VALUES (2, 'B', 'bar');
SELECT * FROM people;

- Error will be thrown because sequence for id value is still 2 and there is a record which id is 2

In [ ]:
%%sql 
INSERT INTO people (name, address) VALUES ('C', 'baz');

##### d. Example usage of <i>GENERATED BY DEFAULT without UNIQUE descriptor</i>

- Creating table with GENERATED BY DEFAULT AS IDENTITY column

In [ ]:
%%sql
DROP TABLE IF EXISTS people;
CREATE TABLE people (
	id bigint GENERATED BY DEFAULT AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

- First record will be inserted successfully and id will be automatically assigned as 1.
- Second record will be inserted successfully.

In [ ]:
%%sql 
INSERT INTO people (name, address) VALUES ('A', 'foo');
INSERT INTO people (id, name, address) VALUES (2, 'B', 'bar');
SELECT * FROM people;

- Insert will be successful but since sequence for id value is still 2 and there is a record which id is 2, there is two record which has id 2.

In [ ]:
%%sql 
INSERT INTO people (name, address) VALUES ('C', 'baz');
SELECT * FROM people;

##### e. Usage of DEFAULT value

- DEFAULT also can be used to specify the sequence explicitly.
- Especially in UPDATE command when update generated column value and get it from sequence, you can use DEFAULT.

- Creating table has generated column

In [ ]:
%%sql
DROP TABLE IF EXISTS people;
CREATE TABLE people (
	id bigint GENERATED ALWAYS AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

- Inserting values.
- When inserting values id will be fetched from a sequence.
- First record has id 1, and second has id 2.
- After insertions completed, current value of sequence will be 3.

In [ ]:
%%sql
INSERT INTO people (name, address) VALUES ('A', 'foo');
INSERT INTO people (name, address) VALUES ('B', 'bar');
SELECT * FROM people;

- Inserting value using DEFAULT.
- Using DEFAULT means use sequence to insert value.
- So the value will be current value of sequence (3) and current value of sequence increases.

In [ ]:
%%sql 
INSERT INTO people (id, name, address) VALUES (DEFAULT, 'C', 'baz');
SELECT * FROM people;

- DEFAULT value also can be used in UPDATE statement.
- When using update statement, id value will be fetched from sequence still.
- So new id of third record will be 4.
- The other use of DEFAULT is in name column. Default value of name is NULL because it is not specified a default value.

In [ ]:
%%sql
UPDATE people SET id=DEFAULT, name=DEFAULT, address='qwe' where id = 3;
SELECT * FROM people; 

### 5.4. Generated Columns

- Syntax is GENERATED [ALWAYS|BY DEFAULT] AS ... STORED
- Creates a calculated column by using a column as base column.
- Example usages:
    - When you have to calculate a value using a column frequently, then you can store the calculated value in another column.
        - By that way you can also index that column and make optimizations.
    - When calculating, if base column is NULL then generated column will be also NULL.
    - (From documentation) Generated columns maintain access privileges separately from their underlying base columns.
        - So, it is possible to arrange it so that a particular role can read from a generated column but not from the underlying base columns.
        - For example you can hide email value from users except current user is admin (for simplicity current_user used instead of role based query).
            - To achive this there are 2 columns for email: email and visible_email
            - visible_email will be generated using email.
            - When getting visible_email if current user is admin email will be seen, but is not admin ***** will be seen in results.
            - To prevent other users get email by using email column, user rights will be revoked.
            - ```sql
                CREATE TABLE users (
                    id SERIAL PRIMARY KEY,
                    email VARCHAR(200) NOT NULL,
                    visible_email VARCHAR(200) GENERATED ALWAYS AS (
                        CASE 
                            WHEN current_user = 'admin' THEN email
                            ELSE '*****'
                        END
                    ) STORED
                );
                REVOKE SELECT ON users FROM PUBLIC;
                GRANT SELECT (id, visible_email) ON users TO PUBLIC;
                GRANT SELECT (id, email, visible_email) ON users TO admin;
              ```
     

##### a. Creating table with GENERATED column

In [ ]:
%%sql
DROP TABLE IF EXISTS people;
CREATE TABLE people (
	id bigint PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
	name varchar(80),
	address varchar(80),
	height_cm numeric,
	height_in numeric GENERATED ALWAYS AS (height_cm / 2.54) STORED
);

- Inserting values to table with GENERATED column.
- See that when height_cm is null, generated column is also null.

In [ ]:
%%sql
INSERT INTO people (name, address, height_cm) VALUES ('A', 'foo', 170); 
INSERT INTO people (name, address) VALUES ('B', 'bar');
SELECT * FROM people;

### 5.5. Constraints

#### 5.5.1. Check Constraints

##### a. Creating table with check constraint

In [ ]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CHECK (price > 0)
);

- Inserting record to table has check constraint
- Operation will throw error because of price should be greater than zero.

In [ ]:
%sql INSERT INTO products VALUES (1, 'product-1', 0);

##### b. Creating table with check constraint which has name

- Giving a constraint name clarifies error messages and allows you to refer it when constraint needed to change

In [ ]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CONSTRAINT positive_price CHECK (price > 0)
);

- Inserting record to table has named check constraint
- Operation will throw error because of price should be greater than zero.
- See that error message includes constraint name (positive_price).

In [ ]:
%sql INSERT INTO products VALUES (1, 'product-1', 0);

##### c. Removing constraint from table and adding new constraint

- First creating table with constraint and give a name to constraint.

In [ ]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CONSTRAINT positive_price CHECK (price > 0)
);

- Removing a constraint and adding new constraint.
- Since constraint has a name it easy to remove using name.
- When not given a name you should find constraint int pg tables. (For more: Google -> PostgreSQL drop constraint with unknown name)

In [ ]:
%%sql
ALTER TABLE products DROP CONSTRAINT positive_price;
ALTER TABLE products ADD CONSTRAINT minimum_price CHECK (price >= 10.0);

##### d. Other constraint informations

- Check constraint can be added for multiple columns or full table.
- When adding new constraint to non-empty table, all records should complain new constraint. Otherwise constraint is violated by some row error will be thrown. (This is not always the case; details will be provided later.)

- First recreating table for cleaning constraints.

In [ ]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CONSTRAINT positive_price CHECK (price > 0)
);

- Adding constraint using multiple columns.

In [ ]:
%sql ALTER TABLE products ADD CONSTRAINT product_no_and_price CHECK (product_no > 0 and price > 0);

- Adding constraint for all columns.

In [ ]:
%sql ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false);

- Check constraing can be applied only new added or modified records.
- To achieve this NOT VALID should be used.

- First cleaning constraints.

In [ ]:
%%sql 
ALTER TABLE products DROP CONSTRAINT IF EXISTS product_no_and_price;
ALTER TABLE products DROP CONSTRAINT IF EXISTS lock_table;

- Inserting values to product table to see effect of new constraints on old values.

In [ ]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 20);
SELECT * FROM products;

- First see when adding new constraing without NOT VALID property.
- It throws error because there are rows which not comply with new constraint.

In [ ]:
%sql ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false);

- Now when using NOT VALID, constraint does not check old values
- So there will be no error thrown.

In [ ]:
%sql ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false) NOT VALID;

- You can check the constraint for old values manually.
- To do that you can use VALIDATE command.
- This statement throws error if there is any record that violates new constraint.

In [ ]:
%sql ALTER TABLE products VALIDATE CONSTRAINT lock_table;

- Also when inserting new value to table it will throw error.
- Because constraint does not allow new records.

In [ ]:
%sql INSERT INTO products VALUES (3, 'product-3', 30);

#### Constraint property DEFERRABLE (Independent from book)

- For constraints there is also DEFERRABLE property is available.
- DEFERRABLE specifies action timing for constraints.
- It specifies will constraint applied immediately or applied end of transaction.
- Usage is: CONSTRAINT ... [ DEFERRABLE | NOT DEFERRABLE ] [ INITIALLY DEFERRED | INITIALLY IMMEDIATE ]
    - [ DEFERRABLE | NOT DEFERRABLE ]: default value is NOT DEFERRABLE.
    - [ INITIALLY DEFERRED | INITIALLY IMMEDIATE ]: default value is INITIALLY IMMEDIATE.
- INITIALLY DEFERRED means in all operations constraint will be checked immediately before insert/update.
- INITIALLY DEFERRED means checking constraint wil be deferred until end of transaction (commit, rollback or error states).
- Actually this cases will be initial case for DEFERRABLE. That means you can change DEFERRED or IMMEDIATE status in the transaction. But this change is only valid in transaction. Does not affect other transactions.
    - For example when defining foreign key constraint with DEFERRABLE without specifying immediate or deferred, its default value is INITIALLY IMMEDIATE.
        - SET CONSTRAINT ... DEFERRED
    - So when in transaction if foreign key will be given later in the transaction, you can set status to DEFERRED and foreign key constraint will be checked end of transaction.
        - SET CONSTRAINT ... IMMEDIATE
    - The reverse could have also been done. A constraint can be defined INITIALLY DEFERRED and you may want to see result of an operation without waiting end of transaction. So you can change it to IMMEDIATE in the transaction. Then constraint will be checked immediately.
- If NOT DEFERRABLE used, you can not change the defer status in the transaction temporarily.
- Yes it can be changed by using ALTER TABLE statement but if affects all latter transactions. Since this action involves modifying the database schema, it is not recommended to use it indiscriminately.
    - ALTER TABLE ... DROP CONSTRAINT ...;
    - ALTER TABLE ... ADD CONSTRAINT ... FOREIGN KEY (...) REFERENCES ... DEFERRABLE INITIALLY DEFERRED;

- To see the effects in this section auto-commit will be disabled.
- Do not forget to re-open for other sections.

In [ ]:
%config SqlMagic.autocommit=False

###### e.1 DEFERRABLE NOT DEFERRABLE usage

- Since default value for deferration is NOT DEFERRABLE, unique_product_no constraint will be NOT DEFERRABLE 

In [ ]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE,
	name text,
	price numeric
);
COMMIT;

- Second insertion will throw error immediately without waiting the transaction ends. 
- Since error thrown in transaction, it rolls back and none of the insertions will be reflected in the table.

In [ ]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 20);
COMMIT;

In [ ]:
%sql SELECT * FROM products;

###### e.2 DEFERRABLE INITIALLY DEFERRED usage

- Creating table with INITIALLY DEFERRED statement.

In [ ]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE DEFERRABLE INITIALLY DEFERRED,
	name text,
	price numeric
);
COMMIT;

- In this transaction second insertion will not throw error. Because unique constraint check deferred until transaction ends.
- In third statement products with id 1 are deleted.
- And in fourth statement a product with id 1 is inserted to empty table.
- At the end of transaction there is only 1 record which has id 1.
- So transaction commit successfully because all records comply with unique constraint.

In [ ]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (1, 'product-1', 10);
DELETE FROM products where product_no=1;
INSERT INTO products VALUES (1, 'product-1', 10);
COMMIT;

- In the end there is only 1 record in table.

In [ ]:
%sql SELECT * FROM products;

- An example that gives an error.
- But the error will be thrown at the end of transaction.

In [ ]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (1, 'product-1', 10);
COMMIT;

###### e.3 Changing DEFERRABLE status in transaction

- First creating a table with DEFERRABLE INITIALLY IMMEDIATE

In [ ]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE DEFERRABLE INITIALLY IMMEDIATE,
	name text,
	price numeric
);
COMMIT;

- In this transaction after adding a record to table changed CONSTRAINT status to DEFERRED.
- When trying to add new record with same id do not throw error, because constraint deferred until transaction ends.
- Since no error thrown, deleting records with id 1, adding new records with id 2 and 1 execute successfully.
- When transaction ends unique constraint check validates that all records.
- Since all records comply with constraint there is 2 record in the table in the end.

In [ ]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
SET CONSTRAINTS unique_product_no DEFERRED;
INSERT INTO products VALUES (1, 'product-1', 10);
DELETE FROM products WHERE product_no = 1;
INSERT INTO products VALUES (2, 'product-2', 20);
INSERT INTO products VALUES (1, 'product-1', 10);
COMMIT;

In [ ]:
%sql SELECT * FROM products;

###### e.4 Changing DEFERRABILITY

- Not specified deferrability so it is NOT DEFERRABLE

In [ ]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE,
	name text,
	price numeric
);
COMMIT;

- Since unique_product_no is not deferrable, when change constraint to DEFERRED will cause error.

In [ ]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
SET CONSTRAINTS unique_product_no DEFERRED;
COMMIT;

- To make NOT DEFERRABLE constraint to DEFERRED, ALTER TABLE used.
- This is not temporarily in transaction and affects all latter transactions.

In [ ]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
ALTER TABLE products DROP CONSTRAINT unique_product_no;
ALTER TABLE products ADD CONSTRAINT unique_product_no UNIQUE(product_no) DEFERRABLE INITIALLY DEFERRED;
INSERT INTO products VALUES (1, 'product-1', 10);
DELETE FROM products WHERE product_no = 1;
INSERT INTO products VALUES (2, 'product-2', 20);
INSERT INTO products VALUES (1, 'product-1', 10);
COMMIT;

In [ ]:
%sql SELECT * FROM products;